## Human Activity Recognition using LSTM

LSTM based activity recognition on the raw data found [here](https://www.cis.fordham.edu/wisdm/dataset.php#actitracker)

### 0. Setup 

In [53]:
!pip install numpy pandas arff tensorflow

In [74]:
import numpy as np
import pandas as pd
import statistics

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras import Sequential

### 1. Load Data

In [75]:
raw_data = pd.read_csv('dataset/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt',names=["user","activity", "timestamp","x","y", "z"], on_bad_lines='skip', lineterminator=';')

/var/folders/13/qyc33pbx2ksbmfc_4_r98v9m0000gn/T/ipykernel_65745/2108347184.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('dataset/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt',names=["user","activity", "timestamp","x","y", "z"], on_bad_lines='skip', lineterminator=';')


In [76]:
raw_data

,user,activity,timestamp,x,y,z
0,33,Jogging,4.910596e+13,-0.694638,12.680544,0.503953
1,33,Jogging,4.910606e+13,5.012288,11.264028,0.953424
2,33,Jogging,4.910611e+13,4.903325,10.882658,-0.081722
3,33,Jogging,4.910622e+13,-0.612916,18.496431,3.023717
4,33,Jogging,4.910633e+13,-1.184970,12.108489,7.205164
...,...,...,...,...,...,...
1086470,\r\n19,Sitting,1.316234e+14,9.040000,-1.460000,1.730000
1086471,\r\n19,Sitting,1.316234e+14,9.080000,-1.380000,1.690000
1086472,\r\n19,Sitting,1.316235e+14,9.000000,-1.460000,1.730000
1086473,\r\n19,Sitting,1.316235e+14,8.880000,-1.330000,1.610000


### 2. Data transformation

Reshape the data into segments to a 3D shape with first dimention representing an example, second dimention represeting timesteps (sequences) and the third the 3 axis reading per time step

In [77]:
STEP = 20
TIME_STEPS = 200
FEATURES = 3
segments = []
labels = []
for i in range(0, len(raw_data) - TIME_STEPS, STEP):
    xs = np.asarray(raw_data['x'][i:i + TIME_STEPS])
    ys = np.asarray(raw_data['y'][i:i + TIME_STEPS])
    zs = np.asarray(raw_data['z'][i:i + TIME_STEPS])
    label = statistics.mode(raw_data['activity'][i:i + TIME_STEPS])[0][0]
    segments.append([xs,ys,zs])
    labels.append(label)    
segments_processed = np.asarray(segments, dtype = np.float32).reshape(-1, TIME_STEPS, FEATURES)
labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

In [78]:
segments_processed.shape

(54314, 200, 3)

In [79]:
segments_processed[1]

array([[  3.568531  ,  -0.50395286,  -2.3018389 ],
       [ -3.568531  ,  -0.8036005 ,   0.50395286],
       [  5.706926  ,  -8.662541  ,  -1.334794  ],
       [ -4.5900574 ,   3.8681788 ,  -1.7978859 ],
       [  7.668256  ,  -2.3699405 ,   2.7240696 ],
       [ -3.5957718 ,   3.9499009 ,   0.46309182],
       [  3.7864566 ,   3.336985  ,   5.6660647 ],
       [  0.23154591,  -4.8216033 ,   1.8387469 ],
       [ -3.2961242 ,  11.604536  ,  -3.173541  ],
       [  0.61291564,  -7.8861814 ,   3.1463003 ],
       [ -3.0237172 ,  -0.08172209,  -1.0351465 ],
       [ -2.4516625 ,  -1.3756552 ,  -2.4925237 ],
       [  1.4573772 ,  -1.4165162 ,  -1.2666923 ],
       [ -3.6774938 ,   1.8387469 ,  -1.2666923 ],
       [  2.4108016 ,  -0.61291564,   0.04086104],
       [ -7.6546354 ,  -1.2666923 ,  -5.012288  ],
       [  1.9477097 ,  -5.053149  ,   9.384419  ],
       [ -5.434519  ,  -0.61291564,   4.7126403 ],
       [ -1.7570249 ,  -0.9125633 ,   2.6014864 ],
       [  5.7886477 ,  -1.98857

In [80]:
labels.shape

(54314, 5)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(segments_processed, labels, test_size=0.2)

### 3. Train

In [82]:
model = Sequential()
model.add(LSTM(128))
model.add(Dense(5))
model.build((1, 200, 3)) 
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (1, 128)                  67584     
                                                                 
 dense_5 (Dense)             (1, 5)                    645       
                                                                 
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


In [83]:
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

In [84]:
model.fit(X_train, y_train, batch_size=1024, epochs = 20)

Epoch 1/20
43/43 [==============================] - 37s 814ms/step - loss: 0.1258 - accuracy: 0.5451
Epoch 2/20
43/43 [==============================] - 35s 826ms/step - loss: 0.0816 - accuracy: 0.7205
Epoch 3/20
43/43 [==============================] - 35s 809ms/step - loss: 0.0513 - accuracy: 0.8231
Epoch 4/20
43/43 [==============================] - 35s 804ms/step - loss: 0.0447 - accuracy: 0.8383
Epoch 5/20
43/43 [==============================] - 36s 839ms/step - loss: 0.0395 - accuracy: 0.8585
Epoch 6/20
43/43 [==============================] - 35s 810ms/step - loss: 0.0374 - accuracy: 0.8672
Epoch 7/20
43/43 [==============================] - 35s 808ms/step - loss: 0.0358 - accuracy: 0.8748
Epoch 8/20
43/43 [==============================] - 35s 820ms/step - loss: 0.0346 - accuracy: 0.8815
Epoch 9/20
43/43 [==============================] - 35s 813ms/step - loss: 0.0326 - accuracy: 0.8894
Epoch 10/20
43/43 [==============================] - 36s 826ms/step - loss: 0.0307 - accura

In [85]:
model.evaluate(X_test, y_test)

340/340 [==============================] - 9s 24ms/step - loss: 0.0220 - accuracy: 0.9293


[0.02204125188291073, 0.9293013215065002]